In [ ]:
!pip install selenium
!pip install webdriver_manager

In [3]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# Function to set up the driver
def setup_driver():
    options = Options()
    options.headless = True  # Set to True for no browser popup
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

def fetch_hotels(search_query):
    driver = setup_driver()
    url = f"https://www.google.com/travel/search?q={search_query}"
    driver.get(url)
    time.sleep(10)  # Wait for the dynamic content to load
    
    # Scroll to load all hotels
    for _ in range(3):  # May need adjustment based on actual page response
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Allow time for lazy-loaded content

    # Prepare to write to CSV
    with open('hotels.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Hotel Name', 'Price'])
        
        # Process each hotel listing
        hotel_listings = driver.find_elements(By.XPATH, "//div[@data-index='4']//a[contains(@class, 'PVOOXe')]")
        count = 0

        for listing in hotel_listings:
            if count >= 15:
                break
            try:
                hotel_name = listing.get_attribute('aria-label')
                price_element = listing.find_element(By.XPATH, ".//following-sibling::div//span[@class='W9vOvb nDkDDb']")
                price = price_element.text
                writer.writerow([hotel_name, price])
                print(f"Hotel: {hotel_name}, Price: {price}")
                count += 1
            except Exception as e:
                print(f"Error extracting data for a hotel: {e}")
    
    driver.quit()

# Usage
search_query = input("Enter location to search hotels: ")
fetch_hotels(search_query)
print("Hotels data has been saved and printed.")


Enter location to search hotels: vellore
Hotels data has been saved and printed.


In [12]:
import requests
import pandas as pd
import re  # Regular expression library for string cleaning

def fetch_hotel_data(search_query):
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google_hotels",
        "q": search_query,
        "gl": "us",
        "hl": "en",
        "currency": "INR",
        "check_in_date": "2024-04-24",
        "check_out_date": "2024-04-25",
        "api_key": "6d3da6b11f95cee42bb81de16eb2b716b1f362601c55f86c9b59c1772bc4bc8f"
    }

    response = requests.get(url, params=params)
    return response.json()

def clean_price(price_str):
    # Removes any character that is not a digit or comma
    cleaned_price = re.sub(r'[^\d,]', '', price_str)
    return cleaned_price

def save_hotels_to_csv(hotel_data):
    hotels_list = []
    if 'properties' in hotel_data:
        for hotel in hotel_data['properties']:
            try:
                if hotel['type'] == 'hotel':  # Ensuring it's a hotel type
                    hotel_name = hotel['name']
                    price_per_night = hotel.get('rate_per_night', {}).get('lowest', 'Not available')
                    cleaned_price = clean_price(price_per_night)  # Clean the price string
                    hotels_list.append({"Hotel Name": hotel_name, "Price Per Night": cleaned_price})
            except KeyError as e:
                print(f"Missing key in data: {e}")
    
    if hotels_list:
        df = pd.DataFrame(hotels_list)
        df.to_csv("hotel_prices.csv", index=False)
        print("Hotel data has been saved to 'hotel_prices.csv'.")
    else:
        print("No hotel data available to save.")

def main():
    user_input = input("Enter your search query for hotels: ")
         
    save_hotels_to_csv(hotel_data)

if __name__ == "__main__":
    main()


Enter your search query for hotels: Kolkata
Hotel data has been saved to 'hotel_prices.csv'.
